In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

%matplotlib inline


In [5]:
med_book_url = 'https://med-books.by/istorii_vnutrennie-bolezni/page/'
king_med_url = 'https://kingmed.info/Istorii-boleznyeĭ'
student_med_url = 'http://studentmedic.ru/history.php?page='


In [116]:
def load_html(url):
    headers = {'user-agent': 'my-app/0.0.1'}
    r = requests.get(url, headers=headers)
    r.encoding = r.apparent_encoding
    return r.text


In [ ]:
med_book_cards = []
for i in range(1, 35):
    next_p = BeautifulSoup(load_html(
        med_book_url + str(i) + '/'), 'html.parser').find_all('div', class_='all')
    med_book_cards.extend(next_p)

len(med_book_cards)
med_book_cards[33]


In [ ]:
med_book_data = [dp.find_all('h2', class_='title')[
    0].text for dp in med_book_cards]
med_book_data = [re.sub(' История болезни: ', '', dp) for dp in med_book_data]
med_book_data[:5]


In [ ]:
med_book_df = pd.DataFrame(med_book_data[1:], columns=['История болезни'])
med_book_df.head()


In [ ]:
med_book_df.to_csv('data/med_book_data.csv', index=False)


In [ ]:
king_med_types = BeautifulSoup(
    load_html(king_med_url), 'html.parser').find_all('div', class_='status')
kind_links = [str(kind.find_all('a')[0].attrs['href'])
              for kind in king_med_types]
king_med_types = []
for link in kind_links:
    url = king_med_url + '/' + link.split('/')[-1]
    html = load_html(url)
    types = BeautifulSoup(html, 'html.parser').find_all('div', class_='status')
    king_med_types.extend(types)

len(king_med_types)


In [ ]:
king_med_data = [kind.text.replace('\n', ' ') for kind in king_med_types]
king_med_df = pd.DataFrame(king_med_data[1:], columns=['История болезни'])
king_med_df.head(20)


In [ ]:
king_med_df.to_csv('data/king_med_data.csv', index=False)


In [ ]:
king_med_df.describe()


In [ ]:
med_book_df.describe()


In [8]:
studentmed_cards = []
for i in range(1, 30):
    next_p = BeautifulSoup(load_html(student_med_url + str(i)),
                           'html.parser').find_all('div', class_='spis')
    for link in next_p:
        for a in link.find_all('ul')[0].find_all('a'):
            studentmed_cards.append(a)
    # studentmed_cards.extend(next_p)

len(studentmed_cards)


724

In [9]:
studentmed_cards[0].attrs['href']
studmed_views = [student_med_url.split('?')[0] + '?' + card.attrs['href'].split(
    '?')[-1].replace('ref', 'view') for card in studentmed_cards]
studmed_views[:5]


['http://studentmedic.ru/history.php?view=434',
 'http://studentmedic.ru/history.php?view=420',
 'http://studentmedic.ru/history.php?view=481',
 'http://studentmedic.ru/history.php?view=395',
 'http://studentmedic.ru/history.php?view=409']

In [11]:
all_divs = []
for view in studmed_views:
    div = BeautifulSoup(load_html(view), 'html.parser').find_all(
        'div', class_='main')[0]
    all_divs.append(div)


In [214]:
from tools.text_tools import process_ru

studmed_data = []
for div in all_divs:
    all_is = []
    for p in div.find_all('p'):

        for i in p.find_all('i'):
            t: str = i.text.replace('\n', '').replace(
                '\t', '').replace('"', '').replace(';', '').strip()
            all_is.append(t)

    sanitaized = process_ru(' '.join(all_is).strip())
    if sanitaized and len(sanitaized) > 128:
        studmed_data.append(sanitaized)

len(studmed_data)


281

In [215]:
studmed_df = pd.DataFrame(studmed_data, columns=['История болезни'])
studmed_df.head(10)


,История болезни
0,Московская Медицинская Академия им. И. М. Сече...
1,ИСТОРИЯ БОЛЕЗНИ Подвижность легочного края Выс...
2,заведующий кафедрой доцент преподаватель ассис...
3,Паспортные данные Жалобы 1. При поступлении 2....
4,"53 года г. Москва электромонтажник <DATE> г., ..."
5,Дыхательная система. Пальпация. Перкуссия. Топ...
6,Клинический диагноз аденома гипофиза 1. Физиче...
7,"опущение матки Аденомиоз, эндометриоз перешейк..."
8,Наследственность и семейный анамнез. История ж...
9,Жалобы больного на день курации 3. Дополнитель...


In [216]:
studmed_df.describe()


,История болезни
count,281
unique,279
top,с. наружная конъюгата истинная конъюгата Ромб ...
freq,2


In [217]:
studmed_df['История болезни'].replace('', np.nan, inplace=True)
studmed_df.dropna(inplace=True)
studmed_df.drop_duplicates(inplace=True)
studmed_df.head(10)


,История болезни
0,Московская Медицинская Академия им. И. М. Сече...
1,ИСТОРИЯ БОЛЕЗНИ Подвижность легочного края Выс...
2,заведующий кафедрой доцент преподаватель ассис...
3,Паспортные данные Жалобы 1. При поступлении 2....
4,"53 года г. Москва электромонтажник <DATE> г., ..."
5,Дыхательная система. Пальпация. Перкуссия. Топ...
6,Клинический диагноз аденома гипофиза 1. Физиче...
7,"опущение матки Аденомиоз, эндометриоз перешейк..."
8,Наследственность и семейный анамнез. История ж...
9,Жалобы больного на день курации 3. Дополнитель...


In [218]:
studmed_df.to_csv('data/studmed_data.csv', index=False)


In [219]:
studmed_df.describe()


,История болезни
count,279
unique,279
top,Московская Медицинская Академия им. И. М. Сече...
freq,1


In [220]:
med_book_df = pd.read_csv('data/med_book_data.csv')
king_med_df = pd.read_csv('data/king_med_data.csv')


In [221]:
def process_df(df):
    df['История болезни'] = df['История болезни'].apply(
        lambda x: process(x).strip())
    df['История болезни'].replace('', np.nan, inplace=True)

    df["Count"] = df['История болезни'].apply(lambda x: len(x))
    df = df[(df["Count"] >= 128)]
    df = df.drop(columns=['Count'])

    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    return df


med_book_df = process_df(med_book_df)
med_book_df.describe()


,История болезни
count,42
unique,42
top,ИБС. Нестабильная стенокардия А класса по Брау...
freq,1


In [222]:
king_med_df = process_df(king_med_df)
king_med_df.describe()


,История болезни
count,317
unique,317
top,"беременность, 3738 недель. Возрастная первород..."
freq,1


In [223]:
all_data = studmed_df.append(med_book_df, ignore_index=True)
all_data.append(king_med_df, ignore_index=True).to_csv(
    'data/all_data.csv', index=False)


/var/folders/hm/j4vjw44n4k99t0758zmc1h0r0000gn/T/ipykernel_87192/10922732.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = studmed_df.append(med_book_df, ignore_index=True)
/var/folders/hm/j4vjw44n4k99t0758zmc1h0r0000gn/T/ipykernel_87192/10922732.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data.append(king_med_df, ignore_index=True).to_csv('data/all_data.csv', index=False)
